# Introduction

---

Links to notebooks in this repository:

[Quickstart Tutorial](./quickstart_tutorial.ipynb) | [Introduction](./00_introduction.ipynb) | [Services](./01_services.ipynb) | [Sleep Staging](./02_sleep_staging.ipynb) | [Ensembling Sleep Staging](./03_ensembling_sleep_staging.ipynb) | [Sleep Dynamics](./04_sleep_dynamics.ipynb) | [Luna Toolbox Integration](./05_luna_integration.ipynb)

---

Welcome to **SLEEPYLAND**, a Python-based toolkit mainly designed to provide advanced sleep data analysis using state-of-the-art deep learning algorithms. The platform enables users to analyze large datasets from the [National Sleep Research Resource (NSRR)](https://sleepdata.org/) and supports sleep staging tasks with automated machine learning models.

<br>

> ### How to download and run SLEEPYLAND
> The instructions for downloading and running SLEEPYLAND are provided in the [`README.md`](./../../../README.md) file located in the repository and in our [SLEEPYLAND YouTube playlist](https://www.youtube.com/playlist?list=PLl7J6NiydmVKJ2SAh7fTKaSK-JoPTgvG-). Simply follow the step-by-step installation pipeline to set up SLEEPYLAND on your machine. The tool is containerized via Docker Compose, ensuring easy deployment and management.

In [ ]:
from IPython.display import Image, display
display(Image(filename='./../../../resources/imgs/sleepyland_architecture.png'))

## Architecture & Development

---

The image above illustrates how SLEEPYLAND's components/services interact with each other. The SLEEPYLAND architecture is designed to provide an easy-to-use, scalable solution for sleep data analysis, integrating several machine learning tools and interfaces.

<br>

### 🔗 **Docker Services**

SLEEPYLAND has been developed using **Docker Compose**, with each key service running in a dedicated container to ensure modularity and scalability. All Docker images and containers are stored on a [**Docker Hub**](https://hub.docker.com/r/bspsupsi/sleepyland), making it easy to pull and run them on your local machine. By following the instructions in the `README.md`, you can easily run the executable script on Linux, Mac or Windows, automatically setting up everything in the background.

<br>

#### Why Docker Compose?

Using Docker Compose to orchestrate multiple containers offers several key advantages in that specific context/application:

| **Advantage**       | **Description**                                                                                                                                                                                                        |
|---------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Isolation & Modularity | Each service (`gui`, `manager-api`, `usleepyland`, etc.) runs in its own dedicated container. This reduces dependency conflicts and allows each service to be updated, maintained, or scaled independently. |
| Ease of Deployment | Docker Compose simplifies deployment, allowing you to bring up the entire SLEEPYLAND environment with one command (`docker-compose up`), minimizing the complexity of managing multiple services.                      |
| Reproducibility   | Docker ensures consistency across environments, eliminating the "it works on my machine" issue. SLEEPYLAND will run the same way on any system, regardless of the OS or machine configuration.                         |
| Scalability       | The architecture supports horizontal scalability. You can easily scale services (e.g., `usleepyland`) with Docker Compose (`docker-compose up --scale usleepyland=3`) to handle larger workloads.                      |
| Cross-Platform Support | Docker Compose runs consistently on multiple operating systems, ensuring SLEEPYLAND can be deployed seamlessly across Windows, macOS, and Linux environments.                                                          |

<br>

#### How it works in the backend?

Below is a structured breakdown of how these services work in the backend:

| **Service**           | **Description**                                                                                                                                                                                                                                                      | **Docker Image**                    | **Port** |
|-----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------|----------|
| **``gui``**           | A Flask-based web interface for user interaction, allowing users to upload/download data, trigger sleep analysis tasks, and view results.                                                                                                                            | `bspsupsi/sleepyland:gui`           | `8887`   |
| **``manager-api``**   | Orchestrates communication between services such as ``gui``, ``usleepyland`` (i.e., pre-trained models), enabling smooth integration between services.                                                                                                                                      | `bspsupsi/sleepyland:manager-api`   | `8989`   |
| **``notebook``**      | A Jupyter notebook interface that allows users to interact with the NSRR sleep data, with the Luna Toolbox, running custom analyses and generating sleep reports.                                                                                                                                | `bspsupsi/sleepyland:notebook`      | `8888`   |
| **``nsrr-download``** | Automates the download of sleep data from the NSRR, enabling users to select cohorts and subjects/recordings for analysis.                                                                                                                                           | `bspsupsi/sleepyland:nsrr-download` | `8500`   |
| **``usleepyland``**   | Hosts feature-based model (e.g., YASA, POPS) and deep learning models (e.g., U-Sleep, DeepResNet, SleepTransformer, L-SeqSleepNet) for automated sleep staging based on EEG and/or EOG signals.                                                                          | `bspsupsi/sleepyland:usleepyland`   | `7777`   |
| **``wild-to-fancy``** | Handles the data harmonization process, converting sleep recordings from `.edf` to `.h5`, resampling data (eventually extracting relevant signals for further analysis), converting annotations from a list of supported formats (e.g., `.xml`, `.annot`) to `.ids`. | `bspsupsi/sleepyland:wild-to-fancy` | `6666`   |

<br>

### 🔗 **Docker Volumes and Directories**

SLEEPYLAND uses **Docker volumes** and **mounted directories** to ensure data consistency and persistence across containers:

| **Type**            | **Name**        | **Purpose**                                                                                                  |
|---------------------|-----------------|--------------------------------------------------------------------------------------------------------------|
| **Volume**          | `input`         | Stores uploaded/downloaded data, shared among all services for consistent input across all tasks.            |
| **Volume**          | `output`        | Holds processed results from, e.g., sleep staging, and other analysis tools.                                 |
| **Host Dir**  | `./my-notebook` | Mapped to the notebook container for users to save custom notebooks and analyses from the Jupyter interface. |

## Interacting with Docker Services in Jupyter

---

This section will guide you through the process of interacting with the different containers/services of SLEEPYLAND using Jupyter Notebook. 

Specifically, we will focus on how to download datasets from NSRR via:

1. **`manager-api`** (as a central interface)
2. Direct communication with the `nsrr-download` container

We'll be sending HTTP requests to these services, both of which are running within the Docker Compose environment.

### Step1 - Downloading NSRR data via the manager-api

In this step, we will send a request to the `manager-api`, which serves as the central interface for various services in SLEEPYLAND, including the data downloading functionality. The manager-api will send the request to the `nsrr-download` service to fetch the data.

Steps:
1. Define the API endpoint for data download in the `manager-api`.
2. Pass an authentication token and specify the datasets you want to download.
3. Send a POST request to the `manager-api` to initiate the download.

In [ ]:
import requests

# Define the Manager API endpoint for downloading data
MANAGER_DOWNLOAD_DATA_URL = "http://manager-api:8989/download_data"

# Replace 'your_token_here' with your actual NSRR token
token = 'your_token_here'

# Specify the edf and annotation paths for the data you want to download
edf_path = "abc/polysomnography/edfs/baseline/abc-baseline-900001.edf"
annotation_path = "abc/polysomnography/annotations-events-nsrr/baseline/abc-baseline-900001-nsrr.xml"

# Combine the edf and annotation paths
data_path = edf_path + "+" + annotation_path

# Specify the datasets to download
datasets = [data_path]

# Send a POST request to the manager-api to download the specified datasets
response = requests.post(MANAGER_DOWNLOAD_DATA_URL, data={'token': token, 'selected_datasets': datasets})

# Handle the response and print the outcome
if response.status_code == 200:
    print("Success:", response.json())
else:
    print(f"Failed with status code {response.status_code}")

### Step2 - Downloading NSRR data via the nsrr-download

In this step, we will interact directly with the `nsrr-download` container. By bypassing the `manager-api`, we can send a request directly to the service responsible for downloading datasets from NSRR.

Steps:

1. Define the API endpoint for the `nsrr-download` container.
2. Pass an authentication token and specify the datasets you want to download.
3. Send a POST request directly to the `nsrr-download` service.

In [ ]:
import requests

# Define the NSRR Download service API endpoint
NSRR_DOWNLOAD_DATA_URL = "http://nsrr-download:8500/download_data"

# Replace 'your_token_here' with your actual NSRR token
token = 'your_token_here'

# Specify the edf and annotation paths for the data you want to download
edf_path_1 = "abc/polysomnography/edfs/baseline/abc-baseline-900001.edf"
annotation_path_1 = "abc/polysomnography/annotations-events-nsrr/baseline/abc-baseline-900001-nsrr.xml"

# Specify the second set of data (edf and annotation paths) to download
edf_path_2 = "abc/polysomnography/edfs/baseline/abc-baseline-900002.edf"
annotation_path_2 = "abc/polysomnography/annotations-events-nsrr/baseline/abc-baseline-900002-nsrr.xml"

# Combine the edf and annotation paths for each set of data
data_path_1 = edf_path_1 + "+" + annotation_path_1
data_path_2 = edf_path_2 + "+" + annotation_path_2

# Specify the datasets to download (combine edf and annotation paths)
datasets = [data_path_1, data_path_2]

# Send a POST request directly to the NSRR Download API to download the datasets
response = requests.post(NSRR_DOWNLOAD_DATA_URL, data={'token': token, 'selected_datasets': datasets})

# Handle the response and print the outcome
if response.status_code == 200:
    print("Success:", response.json())
else:
    print(f"Failed with status code {response.status_code}")

### Explanation of the steps above
> 
> Let's break down what happens during the processes above:
> #### *API Endpoints*:
> - Step 1: We use the endpoint `http://manager-api:8989/download_data`. This is the `manager-api`, which orchestrates communication between the user and the various services within SLEEPYLAND. It serves as a centralized point for interactions, and it forwards the request to the appropriate services like `nsrr-download`.
> - Step 2: We interact directly with the `nsrr-download` container using the API endpoint `http://nsrr-download:8500/download_data`. This bypasses the `manager-api` and communicates directly with the download service for more control or debugging.
> #### *Authorization Token*:
> - In both steps, we pass an NSRR token (`your_token_here`). This token is required to authenticate your request with the NSRR to allow access to the datasets.
> - You must replace `'your_token_here'` with your actual NSRR token in the requests. Without a valid token, you will not be able to access the datasets.
> #### *Datasets*:
> - The `datasets` variable contains the file paths for the datasets you want to download. The format combines the EDF file and its corresponding annotations file, allowing the service to retrieve both the sleep recording and its associated annotations.
> - Multiple datasets can be specified for bulk download by listing each dataset in the `datasets` variable.
> #### *POST Request*:
> - We use a `POST` request to send the token and dataset list to the API endpoint (either `manager-api` or `nsrr-download`). The server processes the request and initiates the download of the specified datasets.


## Explore more: SLEEPYLAND Services

To dive deeper into the available services and the implemented main options for each service within SLEEPYLAND, head over to the SLEEPYLAND Services notebook. This notebook provides detailed information on how each service operates, the options available, and how to interact with them.

👉 [Services](./01_services.ipynb)

> From this notebook onward, all interactions with the different services (such as `nsrr-download`, `usleepyland`, etc.) will be handled via the `manager-api`. The `manager-api` serves as a central communication point, making it easier to manage the requests between services. This approach ensures:
> - Simplified Communication: by using the `manager-api`, you only need to send requests to one service, which then communicates with the appropriate container (e.g., `nsrr-download`, `usleepyland`).
> - Centralized Control: the `manager-api` orchestrates the communication between services in the Docker Compose network, allowing for better coordination of the services without directly exposing each container to external requests.
